In [1]:
import sys
import json
import csv
import random
import time
import torch
import torch.nn as nn
import numpy as np
import fastText
from utils import *
import torch.nn.functional as F


In [2]:
use_en = np.load('data/snips_processed/USE-en.npy')
use_en = np.squeeze(use_en, axis=1)
labs = np.load('data/snips_processed/labs.npy')


dataset, lab2id, id2lab =import_data('data/snips_processed/snips.csv')
sents = dataset[:,2]

In [3]:
sv_model = fastText.load_model('data/cc.sv.300.bin')


In [4]:
def sentence_m(sentence, model, max_len):
    result = []
    words = ['sossos'] + sentence.split() + ['eoseos']
    
    for word in words:
        result.append(model.get_word_vector(word))
    

    if len(words) < max_len:
        result.extend([np.zeros(300)] * (max_len  -  len(words)))
    return np.array(result)
    
    
def prepare_sentences(sents, preprocess=False):       
    sents = pre_process_text(sents)  if preprocess else sents
    lens = list(map(lambda x: x.count(' ') + 3, sents))
    max_len = max(lens)
    m = np.stack(list(map(lambda x:sentence_m(x, sv_model, max_len), sents)))
    return m, lens

In [5]:
data, data_lens = prepare_sentences(sents, True)

In [21]:

class RNN(nn.Module):
    def __init__(self, max_s_len , emb_dim = 300, out_size = 512):
        super(RNN, self).__init__()
        
        #(batch, sent_len, emb_dim)
        
        self.emb_dim = emb_dim
        self.out_size =out_size
        self.max_s_len = max_s_len
        self.kernel_sizes = [2,3,5]
        self.cnn_chan  = 128
        self.lstm_hid  = 256
        
        self.drop = nn.Dropout(0.35)

        self.max_pool_kernel_size = [(self.max_s_len - x + 1, 1) for x in self.kernel_sizes]

        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=self.cnn_chan, kernel_size=(self.kernel_sizes[0], self.emb_dim), )
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=self.cnn_chan, kernel_size=(self.kernel_sizes[1], self.emb_dim), )
        self.conv3 = nn.Conv2d(in_channels=1, out_channels=self.cnn_chan, kernel_size=(self.kernel_sizes[2], self.emb_dim), )
        
        #dont add dropout  to last layer since there is only one layer
        self.lstm = nn.GRU(input_size = self.emb_dim, hidden_size = self.lstm_hid,
                         batch_first=True, bidirectional=True)
    
    
        self.lin = nn.Linear(self.lstm_hid*2 + self.cnn_chan * 3, out_size)
        self.out = nn.LogSoftmax(1)

        
        
    def forward(self, x, lens):
        
        # add chanels dimention:
        xc = x.unsqueeze(1)
        
        x1 = self.conv1(xc)
        x1 = F.max_pool2d(F.relu(x1), self.max_pool_kernel_size[0])
        x1 = x1.squeeze(3).squeeze(2)
        
        x2 = self.conv1(xc)
        x2 = F.max_pool2d(F.relu(x2), self.max_pool_kernel_size[1])
        x2 = x2.squeeze(3).squeeze(2)

        x3 = self.conv1(xc)
        x3 = F.max_pool2d(F.relu(x3), self.max_pool_kernel_size[2])
        x3 = x3.squeeze(3).squeeze(2)

        ps = nn.utils.rnn.pack_padded_sequence(x, lens, batch_first=True, enforce_sorted=False)
        
        _, (h) = self.lstm(ps)
        lstm_out = torch.cat((h[1],h[0]), dim =1)

        z = torch.cat((x1,x2,x3, lstm_out), dim=1)
        
        z = self.drop(z)
        z = self.lin(z)
        
        return self.out(z)
    


In [45]:
def train(model, criterion, optimizer, vectors, lens, labels):
    model.zero_grad()
    model.train()
    loss = 0

    # vectors = torch.tensor(vectors).float()
    # labels = torch.tensor(labels)

    model_out = model.forward(vectors, lens)
    
    loss += criterion(model_out, labels)

    loss.backward()
    optimizer.step()

    return loss.item() / len(labels)

def evaluate(model, vectors, lens,labels,  criterion):
    with torch.no_grad():
        model.eval()
        #vectors = torch.tensor(vectors).float()
        #labels = torch.tensor(labels)
    
        model_out = model.forward(vectors, lens)
        right = 0
        
        for i  in range(len(model_out)):
            k, v = model_out[i].topk(1)
            predicted, true = v.item(), labels[i].item()
            if predicted == true:
                right +=1

                
        loss = criterion(model_out, labels)
        return loss.item(), right/len(model_out)

In [46]:
train_data = data[:10000]
test_data = data[10000:]

train_out = use_en[:10000]
test_out = use_en[10000:]

train_lens = data_lens[:10000]
test_lens = data_lens[10000:]

test_labs = labs[10000:]
train_labs = labs[:10000]

In [47]:
net = RNN(max_s_len=train_data.shape[1], out_size=7)
optimizer = torch.optim.Adam(net.parameters())
criterion = torch.nn.NLLLoss()

vectors = torch.tensor(train_data).float()
labels = torch.tensor(train_labs)

tvectors = torch.tensor(test_data).float()
tlabels = torch.tensor(test_labs)

In [49]:
t = time.time()
for i in range(400):
    loss = train(net, criterion, optimizer, vectors, train_lens, labels)
    if True and not i% 20:
        #print('#{:3d}, {:5d} sec.'.format(i, int(time.time()-t)))
        eval_loss, acc = evaluate(net, tvectors, test_lens, tlabels, criterion)
        print('#{:3d}, {:5d} sec. train loss: {:.7f}, eval loss: {:.4f}, acc = {:.4f}'.format(i, int(time.time() - t), loss, eval_loss, acc))
    

#  0,    25 sec. train loss: 0.0001426, eval loss: 1.3406, acc = 0.6361
# 20,  2706 sec. train loss: 0.0000471, eval loss: 0.4468, acc = 0.8528
# 40,  8971 sec. train loss: 0.0000192, eval loss: 0.2122, acc = 0.9279
# 60,  9403 sec. train loss: 0.0000119, eval loss: 0.1560, acc = 0.9487
# 80,  9842 sec. train loss: 0.0000084, eval loss: 0.1378, acc = 0.9567
#100, 10279 sec. train loss: 0.0000078, eval loss: 0.1289, acc = 0.9585


KeyboardInterrupt: 